In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_df="/kaggle/input/surface-crack/train"
test_df="/kaggle/input/surface-crack/test"
valid_df="/kaggle/input/surface-crack/valid"

In [4]:
train_data = train_gen.flow_from_directory(
    train_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,

 
)

val_data = train_gen.flow_from_directory(
    valid_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True

   

)

test_data = train_gen.flow_from_directory(
    test_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,

    
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,RMSprop, Adagrad

from keras.layers import BatchNormalization

In [6]:
model = Sequential()
model.add(Conv2D(64,3,padding="same", activation="relu", input_shape =(120, 120, 3)))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(128, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(2, activation="softmax"))

model.summary()

2023-01-13 10:08:15.176645: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 10:08:15.177579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 10:08:15.182174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 10:08:15.183123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 10:08:15.183959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [8]:
es = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
callbacks_list = [es]

In [10]:


model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=1e-5), metrics=["accuracy"]) 

history = model.fit(train_data,validation_data=val_data,epochs = 100,callbacks=callbacks_list ,verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2023-01-13 10:09:31.640249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-13 10:09:33.648198: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


19/19 [==============================] - 14s 238ms/step - loss: 0.8146 - accuracy: 0.5250 - val_loss: 0.6884 - val_accuracy: 0.5000
Epoch 2/100
19/19 [==============================] - 2s 95ms/step - loss: 0.7622 - accuracy: 0.5550 - val_loss: 0.6854 - val_accuracy: 0.5000
Epoch 3/100
19/19 [==============================] - 2s 110ms/step - loss: 0.7079 - accuracy: 0.5783 - val_loss: 0.6814 - val_accuracy: 0.5000
Epoch 4/100
19/19 [==============================] - 2s 98ms/step - loss: 0.6622 - accuracy: 0.6100 - val_loss: 0.6769 - val_accuracy: 0.5050
Epoch 5/100
19/19 [==============================] - 2s 96ms/step - loss: 0.6373 - accuracy: 0.6333 - val_loss: 0.6728 - val_accuracy: 0.7300
Epoch 6/100
19/19 [==============================] - 2s 95ms/step - loss: 0.6401 - accuracy: 0.6500 - val_loss: 0.6736 - val_accuracy: 0.7150
Epoch 7/100
19/19 [==============================] - 2s 93ms/step - loss: 0.6005 - accuracy: 0.6617 - val_loss: 0.6683 - val_accuracy: 0.7650
Epoch 8/100
19/

In [11]:
fig = px.line( 
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
)

fig.show()

In [51]:
def evaluate_model(model, val_data):
    
    results = model.evaluate(val_data, verbose=0)
    loss = results[0]
    acc = results[1]
    print("    Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))



In [52]:
evaluate_model(model, test_data)

    Test Loss: 0.55255
Test Accuracy: 69.50%


[1. 0.]
